In [ ]:
import cv2
import numpy as np
import serial
import time

# Load YOLOv4-tiny model
model = cv2.dnn.readNetFromDarknet('yolov4-tiny.cfg', 'yolov4-tiny.weights')
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Get the output layer names
layer_names = model.getLayerNames()
output_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]

# Load the class names from the coco.names file
with open('coco.names.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Open the webcam
#droidcam_ip = 'http://172.20.10.2:4747/video'
# Connect to Droidcam
#cap = cv2.VideoCapture(droidcam_ip)
cap = cv2.VideoCapture(0)


# Set up serial communication
ser = serial.Serial("COM4", 9600)


def sendCommand(command):
    ser.write(command.encode('ascii'))
    time.sleep(0.1)


while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # (the rest of the code remains unchanged)
    # Resize the frame to a size divisible by the network's stride
    net_input_size = (416, 416)  # the network's input size
    stride = 32  # the network's stride
    height, width, _ = frame.shape
    new_height = (height // stride) * stride
    new_width = (width // stride) * stride
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Preprocess the input frame
    blob = cv2.dnn.blobFromImage(
        resized_frame, 1 / 255.0, net_input_size, swapRB=True, crop=False)

    # Set the input to the network and run a forward pass
    model.setInput(blob)
    layer_outputs = model.forward(output_layers)

    # Postprocess the outputs
    boxes = []
    confidences = []
    class_ids = []
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to remove overlapping boxes
    indexes = cv2.dnn.NMSBoxes(
        boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    # Draw the bounding boxes and labels
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))
    person_detected = False
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            color = colors[i]
            class_id = class_ids[i]
            label = str(classes[class_id])
            if class_id == 0:  # Person is detected
                person_detected = True
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 5), font, 1, color, 2)

    #Show the output
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) == ord('q'):
        break

    # Send command to Arduino based on person detectioqqn
        # Send command to Arduino based on person detection
    if person_detected:
        sendCommand('S')  # Stop
        ser.reset_input_buffer()
    else:
        sendCommand('F')  # Forward
        ser.reset_input_buffer()

# Release the resources
cap.release()
cv2.destroyAllWindows()
ser.close()
